# 14주차 과제

# 데이터 세트 준비
아동 건강 및 발달 연구의 데이터세트(R 데이터세트)

In [1]:
import pandas as pd
import statsmodels.api as sm

# 데이터셋 로드
df = sm.datasets.get_rdataset("HairEyeColor", "datasets").data

# 데이터세트 정보
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Hair    32 non-null     object
 1   Eye     32 non-null     object
 2   Sex     32 non-null     object
 3   Freq    32 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.1+ KB


In [2]:
# 첫 5개행
df.head()

,Hair,Eye,Sex,Freq
0,Black,Brown,Male,32
1,Brown,Brown,Male,53
2,Red,Brown,Male,10
3,Blond,Brown,Male,3
4,Black,Blue,Male,11


# 적합성 검정
머리 색상 분포가1:1:1:1 비율을 따르는지 검정

In [3]:
from scipy.stats import chisquare, chi2_contingency

#유의수준
alpha = 0.05

# 데이터 변환: Hair, Eye, Sex를 기준으로 교차표 생성
cross_tab = pd.pivot_table(
    df,                 # 데이터프레임
    values="Freq",     # 값으로 사용할 열
    columns="Hair",    # 열 인덱스
    aggfunc="sum",     # 집계 함수
    fill_value=0       # 결측값을 0으로 채움
    #margins=True,
    # margins_name="sum"
)

print("교차표(Hair):")
display(cross_tab)


교차표(Hair):


Hair,Black,Blond,Brown,Red
Freq,108,127,286,71


In [4]:
# 교차표의 총합
total = cross_tab.loc['Freq'].sum()
# 예상 비율 (각 Hair 색상에 대해 동일한 비율)
expected_ratios = [0.25, 0.25, 0.25, 0.25]
expected = [total * ratio for ratio in expected_ratios]

# 적합성 검정 수행
chi2_stat, p_value = chisquare( f_obs=cross_tab.loc['Freq'], f_exp=expected)

# 결과 출력
print(f"카이제곱 통계량: {chi2_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# 결론 도출
if p_value < alpha:
    print("귀무가설 기각: 머리 색상은 예상 비율과 일치하지 않음")
else:
    print("귀무가설 채택: 머리 색상은 예상 비율과 일치함")


카이제곱 통계량: 182.5270
p-value: 0.0000
귀무가설 기각: 머리 색상은 예상 비율과 일치하지 않음


# 눈색상의 분포가 3.5:4:1:1.5 비율을 따르는지 검정

In [5]:
# 데이터 변환: Eye를 기준으로 교차표 생성
cross_tab = pd.pivot_table(
    df,                 # 데이터프레임
    values="Freq",     # 값으로 사용할 열
    columns="Eye",    # 열 인덱스
    aggfunc="sum",     # 집계 함수
    fill_value=0       # 결측값을 0으로 채움
    #margins=True,
    # margins_name="sum"
)

print("교차표(Eye):")
display(cross_tab)
# 교차표의 총합
total = cross_tab.loc['Freq'].sum()
# 예상 비율 (각 Eye 색상에 대해 동일한 비율)
expected_ratios = [0.35, 0.4, 0.1, 0.15]
expected = [total * ratio for ratio in expected_ratios]

# 적합성 검정 수행
chi2_stat, p_value = chisquare( f_obs=cross_tab.loc['Freq'], f_exp=expected)

# 결과 출력
print(f"카이제곱 통계량: {chi2_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# 결론 도출
if p_value < alpha:
    print("귀무가설 기각: 눈 색상은 예상 비율과 일치하지 않음")
else:
    print("귀무가설 채택: 눈 색상은 예상 비율과 일치함")


교차표(Eye):


Eye,Blue,Brown,Green,Hazel
Freq,215,220,64,93


카이제곱 통계량: 2.0734
p-value: 0.5573
귀무가설 채택: 눈 색상은 예상 비율과 일치함


# 동질성 검정
성별 머리 색상의 분포가 동일한지 검정

In [6]:
# 데이터 변환: Eye를 기준으로 교차표 생성
cross_tab = pd.pivot_table(
    df,                 # 데이터프레임
    values="Freq",     # 값으로 사용할 열
    index="Sex",    # 행 인덱스
    columns="Hair",    # 열 인덱스
    aggfunc="sum",     # 집계 함수
    fill_value=0       # 결측값을 0으로 채움
    #margins=True,
    # margins_name="sum"
)

print("교차표(Sex~Hair):")
display(cross_tab)


교차표(Sex~Hair):


Hair,Black,Blond,Brown,Red
Sex,,,,
Female,52,81,143,37
Male,56,46,143,34


In [7]:
# 동질성 검정 수행
chi2, p_value, dof, expected = chi2_contingency(cross_tab)

# 결과 출력
print(f"기대도수:\n{expected}", end="\n\n")
print("카이제곱 검정 결과:")
print(f"카이제곱 통계량: {chi2:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"자유도: {dof}")

# 결론 도출
alpha = 0.05
if p_value < alpha:
    print("귀무가설 기각: 남성과 여성의 머리 색상 분포는 동일하지 않음")
else:
    print("귀무가설 채택: 남성과 여성의 머리 색상 분포는 동일함")


기대도수:
[[ 57.10135135  67.14695946 151.21283784  37.53885135]
 [ 50.89864865  59.85304054 134.78716216  33.46114865]]

카이제곱 검정 결과:
카이제곱 통계량: 7.9942
p-value: 0.0461
자유도: 3
귀무가설 기각: 남성과 여성의 머리 색상 분포는 동일하지 않음


# 성별 눈 색상의 분포가 동일한지 검정

In [8]:
# 데이터 변환: Eye를 기준으로 교차표 생성
cross_tab = pd.pivot_table(
    df,                 # 데이터프레임
    values="Freq",     # 값으로 사용할 열
    index="Sex",    # 행 인덱스
    columns="Eye",    # 열 인덱스
    aggfunc="sum",     # 집계 함수
    fill_value=0       # 결측값을 0으로 채움
    #margins=True,
    # margins_name="sum"
)

print("교차표(Sex~Eye):")
display(cross_tab)

# 동질성 검정 수행
chi2, p_value, dof, expected = chi2_contingency(cross_tab)

# 결과 출력
print(f"기대도수:\n{expected}", end="\n\n")
print("카이제곱 검정 결과:")
print(f"카이제곱 통계량: {chi2:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"자유도: {dof}")

# 결론 도출
alpha = 0.05
if p_value < alpha:
    print("귀무가설 기각: 남성과 여성의 눈 색상 분포는 동일하지 않음")
else:
    print("귀무가설 채택: 남성과 여성의 눈 색상 분포는 동일함")



교차표(Sex~Eye):


Eye,Blue,Brown,Green,Hazel
Sex,,,,
Female,114,122,31,46
Male,101,98,33,47


기대도수:
[[113.67398649 116.31756757  33.83783784  49.17060811]
 [101.32601351 103.68243243  30.16216216  43.82939189]]

카이제곱 검정 결과:
카이제곱 통계량: 1.5298
p-value: 0.6754
자유도: 3
귀무가설 채택: 남성과 여성의 눈 색상 분포는 동일함


# 독립성 검정
머리 색상과 눈 색상 간의 독립성 검정

In [9]:
# 데이터 변환: Eye를 기준으로 교차표 생성
cross_tab = pd.pivot_table(
    df,                 # 데이터프레임
    values="Freq",     # 값으로 사용할 열
    index="Hair",    # 행 인덱스
    columns="Eye",    # 열 인덱스
    aggfunc="sum",     # 집계 함수
    fill_value=0       # 결측값을 0으로 채움
    #margins=True,
    # margins_name="sum"
)

print("교차표(Hair~Eye):")
display(cross_tab)

# 동질성 검정 수행
chi2, p_value, dof, expected = chi2_contingency(cross_tab)

# 결과 출력
print(f"기대도수:\n{expected}", end="\n\n")
print("카이제곱 검정 결과:")
print(f"카이제곱 통계량: {chi2:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"자유도: {dof}")

# 결론 도출
alpha = 0.05
if p_value < alpha:
    print("귀무가설 기각: 남성과 여성의 눈 색상 분포는 동일하지 않음")
else:
    print("귀무가설 채택: 남성과 여성의 눈 색상 분포는 동일함")



교차표(Hair~Eye):


Eye,Blue,Brown,Green,Hazel
Hair,,,,
Black,20,68,5,15
Blond,94,7,16,10
Brown,84,119,29,54
Red,17,26,14,14


기대도수:
[[ 39.22297297  40.13513514  11.67567568  16.96621622]
 [ 46.12331081  47.19594595  13.72972973  19.95101351]
 [103.86824324 106.28378378  30.91891892  44.92905405]
 [ 25.78547297  26.38513514   7.67567568  11.15371622]]

카이제곱 검정 결과:
카이제곱 통계량: 138.2898
p-value: 0.0000
자유도: 9
귀무가설 기각: 남성과 여성의 눈 색상 분포는 동일하지 않음


# 연습

Titanic 데이터셋

In [19]:
import pandas as pd
import statsmodels.api as sm

# 데이터셋 로드
df = sm.datasets.get_rdataset("Titanic", "datasets").data

# 데이터세트 정보
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Class     32 non-null     object
 1   Sex       32 non-null     object
 2   Age       32 non-null     object
 3   Survived  32 non-null     object
 4   Freq      32 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 1.4+ KB


In [20]:
# 첫 5개행
df.head()

,Class,Sex,Age,Survived,Freq
0,1st,Male,Child,No,0
1,2nd,Male,Child,No,0
2,3rd,Male,Child,No,35
3,Crew,Male,Child,No,0
4,1st,Female,Child,No,0


In [21]:
#적합성 검정
from scipy.stats import chisquare, chi2_contingency

#유의수준
alpha = 0.05

# 데이터 변환: Class, Sex, Age 기준으로 교차표 생성
cross_tab = pd.pivot_table(
    df,                 # 데이터프레임
    values="Freq",     # 값으로 사용할 열
    columns="Class",    # 열 인덱스
    aggfunc="sum",     # 집계 함수
    fill_value=0       # 결측값을 0으로 채움
    #margins=True,
    # margins_name="sum"
)

print("교차표(Class):")
display(cross_tab)


교차표(Class):


Class,1st,2nd,3rd,Crew
Freq,325,285,706,885


In [22]:
# 교차표의 총합
total = cross_tab.loc['Freq'].sum()
# 예상 비율 (각 종 에 대해 동일한 비율)
expected_ratios = [0.25, 0.25, 0.25, 0.25]
expected = [total * ratio for ratio in expected_ratios]

# 적합성 검정 수행
chi2_stat, p_value = chisquare( f_obs=cross_tab.loc['Freq'], f_exp=expected)

# 결과 출력
print(f"카이제곱 통계량: {chi2_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# 결론 도출
if p_value < alpha:
    print("귀무가설 기각: 선실은 예상 비율과 일치하지 않음")
else:
    print("귀무가설 채택: 선실은 예상 비율과 일치함")


카이제곱 통계량: 467.8069
p-value: 0.0000
귀무가설 기각: 선실은 예상 비율과 일치하지 않음


선실 비율이 0.2:0.3:0.2:0.3비율을 따르는지 검정

In [27]:
# 데이터 변환: Class를 기준으로 교차표 생성
cross_tab = pd.pivot_table(
    df,                 # 데이터프레임
    values="Freq",     # 값으로 사용할 열
    columns="Class",    # 열 인덱스
    aggfunc="sum",     # 집계 함수
    fill_value=0       # 결측값을 0으로 채움
    #margins=True,
    # margins_name="sum"
)

print("교차표(Class):")
display(cross_tab)
# 교차표의 총합
total = cross_tab.loc['Freq'].sum()
# 예상 비율 (각 Class 색상에 대해 동일한 비율)
expected_ratios = [0.2, 0.3, 0.2, 0.3]
expected = [total * ratio for ratio in expected_ratios]

# 적합성 검정 수행
chi2_stat, p_value = chisquare( f_obs=cross_tab.loc['Freq'], f_exp=expected)

# 결과 출력
print(f"카이제곱 통계량: {chi2_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# 결론 도출
if p_value < alpha:
    print("귀무가설 기각: 선실은 예상 비율과 일치하지 않음")
else:
    print("귀무가설 채택: 선실은 예상 비율과 일치함")


교차표(Class):


Class,1st,2nd,3rd,Crew
Freq,325,285,706,885


카이제곱 통계량: 480.4198
p-value: 0.0000
귀무가설 기각: 선실은 예상 비율과 일치하지 않음


In [28]:
# 동질성 검정
# 데이터 변환: Sex,Class를 기준으로 교차표 생성
cross_tab = pd.pivot_table(
    df,                 # 데이터프레임
    values="Freq",     # 값으로 사용할 열
    index="Class",    # 행 인덱스
    columns="Sex",    # 열 인덱스
    aggfunc="sum",     # 집계 함수
    fill_value=0       # 결측값을 0으로 채움
    #margins=True,
    # margins_name="sum"
)

print("교차표(Class~Sex):")
display(cross_tab)


교차표(Class~Sex):


Sex,Female,Male
Class,,
1st,145,180
2nd,106,179
3rd,196,510
Crew,23,862


In [29]:
# 동질성 검정 수행
chi2, p_value, dof, expected = chi2_contingency(cross_tab)

# 결과 출력
print(f"기대도수:\n{expected}", end="\n\n")
print("카이제곱 검정 결과:")
print(f"카이제곱 통계량: {chi2:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"자유도: {dof}")

# 결론 도출
alpha = 0.05
if p_value < alpha:
    print("귀무가설 기각: 남성과 여성의 머리 색상 분포는 동일하지 않음")
else:
    print("귀무가설 채택: 남성과 여성의 머리 색상 분포는 동일함")


기대도수:
[[ 69.4002726  255.5997274 ]
 [ 60.85870059 224.14129941]
 [150.75874602 555.24125398]
 [188.98228078 696.01771922]]

카이제곱 검정 결과:
카이제곱 통계량: 349.9145
p-value: 0.0000
자유도: 3
귀무가설 기각: 남성과 여성의 머리 색상 분포는 동일하지 않음


성별, 생존율이 비례하는지 검정

In [31]:
# 데이터 변환: Sex,Survived 기준으로 교차표 생성
cross_tab = pd.pivot_table(
    df,                 # 데이터프레임
    values="Freq",     # 값으로 사용할 열
    index="Sex",    # 행 인덱스
    columns="Survived",    # 열 인덱스
    aggfunc="sum",     # 집계 함수
    fill_value=0       # 결측값을 0으로 채움
    #margins=True,
    # margins_name="sum"
)

print("교차표(Sex~Survived):")
display(cross_tab)

# 동질성 검정 수행
chi2, p_value, dof, expected = chi2_contingency(cross_tab)

# 결과 출력
print(f"기대도수:\n{expected}", end="\n\n")
print("카이제곱 검정 결과:")
print(f"카이제곱 통계량: {chi2:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"자유도: {dof}")

# 결론 도출
alpha = 0.05
if p_value < alpha:
    print("귀무가설 기각: 남성과 여성의 생존률 분포는 동일하지 않음")
else:
    print("귀무가설 채택: 남성과 여성의 생존률 분포는 동일함")



교차표(Sex~Survived):


Survived,No,Yes
Sex,,
Female,126,344
Male,1364,367


기대도수:
[[ 318.17355747  151.82644253]
 [1171.82644253  559.17355747]]

카이제곱 검정 결과:
카이제곱 통계량: 454.4998
p-value: 0.0000
자유도: 1
귀무가설 기각: 남성과 여성의 생존률 분포는 동일하지 않음


독립성검정

In [33]:
# 성별과 나이간의 독립성 검정
# 데이터 변환: Sex,Age 기준으로 교차표 생성
cross_tab = pd.pivot_table(
    df,                 # 데이터프레임
    values="Freq",     # 값으로 사용할 열
    index="Sex",    # 행 인덱스
    columns="Age",    # 열 인덱스
    aggfunc="sum",     # 집계 함수
    fill_value=0       # 결측값을 0으로 채움
    #margins=True,
    # margins_name="sum"
)

print("교차표(Sex~Age):")
display(cross_tab)

# 동질성 검정 수행
chi2, p_value, dof, expected = chi2_contingency(cross_tab)

# 결과 출력
print(f"기대도수:\n{expected}", end="\n\n")
print("카이제곱 검정 결과:")
print(f"카이제곱 통계량: {chi2:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"자유도: {dof}")

# 결론 도출
alpha = 0.05
if p_value < alpha:
    print("귀무가설 기각: 남성과 여성의 나이 분포는 동일하지 않음")
else:
    print("귀무가설 채택: 남성과 여성의 나이 분포는 동일함")



교차표(Sex~Age):


Age,Adult,Child
Sex,,
Female,425,45
Male,1667,64


기대도수:
[[ 446.72421627   23.27578373]
 [1645.27578373   85.72421627]]

카이제곱 검정 결과:
카이제곱 통계량: 25.8905
p-value: 0.0000
자유도: 1
귀무가설 기각: 남성과 여성의 나이 분포는 동일하지 않음
